<div style="background-color: #29261e; border: 1px solid #ffc107; border-left: 4px solid #ffc107; padding: 15px; margin: 10px 0; border-radius: 4px;">
    <strong>⚠️ Work In Progress</strong><br>
    This notebook is currently non-functional due to missing features that are planned for future development.
</div>

# Azure AI Agent with Bing Grounding Example

This notebook demonstrates how to create an Azure AI agent that uses Bing Grounding search to find real-time information from the web.

## Features Covered:
- Setting up Bing Grounding search integration
- Using HostedWebSearchTool for web searches
- Creating agents that can access current web information
- Proper resource management with async context managers

## Prerequisites

Before running this notebook, ensure you have:
- Azure CLI installed and authenticated (`az login --use-device-code`)
- Access to an Azure AI Foundry project with deployed models
- Environment variables set up (see Initial Setup section below)

This example demonstrates Azure AI agents with Bing Grounding for web search capabilities. The agent can search the web for current information and provide informed responses.

## Import Libraries

Import the required libraries for Azure AI agent functionality.

In [ ]:
#!about
#r "nuget: Azure.Identity, 1.18.0-beta.2"
#r "nuget: Microsoft.Agents.AI, 1.0.0-preview.*"
#r "nuget: Microsoft.Agents.AI.AzureAI, 1.0.0-preview.*"
#r "nuget: dotenv.net"

using Azure.Identity;
using Microsoft.Agents.AI;
using Azure.AI.Projects;
using Azure.AI.Projects.OpenAI;
using Microsoft.Extensions.AI;
using dotenv.net;
using System.IO;
using System.Threading.Tasks;
using System.ComponentModel;
using System.Text.Json;

## Initial Setup

Initialize the Azure AI project client and set up authentication.

In [ ]:
DotEnv.Load(new DotEnvOptions(envFilePaths: new[] { Path.Combine(".", "..", "..", ".env") })); 

// Create the Credentials
var credential = new InteractiveBrowserCredential();

AIProjectClient aiProjectClient = new AIProjectClient(
    new Uri(Environment.GetEnvironmentVariable("AZURE_AI_PROJECT_ENDPOINT")!),
    credential);

## Check Environment Setup

Let's verify that the required environment variable is set:

In [ ]:
var bingConnectionId = Environment.GetEnvironmentVariable("BING_CONNECTION_ID")!;

if (string.IsNullOrEmpty(bingConnectionId))
{
    Console.WriteLine("Please set the BING_CONNECTION_ID environment variable in your .env file.");
    return;
}

## Create Bing Grounding Search Tool

The `HostedWebSearchTool` allows our agent to perform web searches using Bing. The connection ID will be automatically picked up from the environment variable:

In [ ]:
var bingSearchTool = AgentTool.CreateBingGroundingTool(new BingGroundingSearchToolParameters(
    [new BingGroundingSearchConfiguration(bingConnectionId)]
));

Console.WriteLine("✅ Successfully created Bing Grounding Tool.");

## Create and Run the Agent

Now let's create an Azure AI agent that can use the Bing search tool to find current information from the web: